In [5]:
import pandas as pd
import nltk
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfTransformer

In [9]:
data = pd.read_excel('csv/data.xlsx')
test = pd.read_csv('csv/dataset.csv')
des = pd.read_csv('csv/symptom_Description.csv')
prec = pd.read_csv('csv/symptom_precaution.csv')
sev = pd.read_csv('csv/Symptom-severity.csv')


In [10]:
column = data.columns
#train
data = data.iloc[0:39]
data = data.fillna('')
data['Disease'].value_counts()

#test
test = test.sample(n = 100) 
test = test.fillna('')

#train
data['Y'] = data[column[1:]].apply(lambda x: ''.join(x), axis = 1) 
#test
test['Y'] = test[column[1:]].apply(lambda x: ''.join(x), axis = 1) 

In [11]:
#train
df = pd.DataFrame()
df['Y'] = data['Disease']
df['X'] = data['Y']
#test
test_df = pd.DataFrame()
test_df['Y'] = test['Disease']
test_df['X'] = test['Y']

#train
df['Y'] = df['Y'].str.lower()
df['X'] = df['X'].str.lower()
df['X'] = df['X'].str.strip()
df['X'] = df['X'].str.replace('and' ,'')
df['X'] = df['X'].str.replace('_' , ' ')

#test
test_df['Y'] = test_df['Y'].str.lower()
test_df['X'] = test_df['X'].str.lower()
test_df['X'] = test_df['X'].str.strip()
test_df['X'] = test_df['X'].str.replace('and' ,'')
test_df['X'] = test_df['X'].str.replace('_' , ' ')

In [12]:
def lemma (x):
    WNlemma = nltk.WordNetLemmatizer()
    return ([WNlemma.lemmatize(t) for t in x])
def stem(x):
    porter = nltk.PorterStemmer()
    return ([porter.stem(t) for t in x])
def listToString(s):   
    return( ' '.join(s) )

#train
df['X'] = df['X'].apply(lambda x: nltk.word_tokenize(x))
df['X'] = df['X'].apply(lambda x: stem(x))
df['X'] = df['X'].apply(lambda x: lemma(x))
df['X'] = df['X'].apply(lambda x: listToString(x))

#test
test_df['X'] = test_df['X'].apply(lambda x: nltk.word_tokenize(x))
test_df['X'] = test_df['X'].apply(lambda x: stem(x))
test_df['X'] = test_df['X'].apply(lambda x: lemma(x))
test_df['X'] = test_df['X'].apply(lambda x: listToString(x))

X = df.X
y = df.Y
X_test = test_df.X
y_test = test_df.Y

In [13]:
NN = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('mlpclf', MLPClassifier(hidden_layer_sizes = [100,100], solver = 'adam', random_state = 0)),
               ])
NN.fit(X,y)
y_pred = NN.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names= test_df.Y.unique()))


accuracy 0.93
                                         precision    recall  f1-score   support

                        osteoarthristis       1.00      1.00      1.00         4
                                   acne       1.00      1.00      1.00         3
(vertigo) paroymsal  positional vertigo       0.00      0.00      0.00         4
                            hepatitis c       1.00      1.00      1.00         4
                                   gerd       1.00      1.00      1.00         4
                                   aids       1.00      1.00      1.00         4
                        hyperthyroidism       1.00      1.00      1.00         2
                          hypertension        1.00      1.00      1.00         3
                   cervical spondylosis       0.50      1.00      0.67         2
                                 dengue       1.00      1.00      1.00         3
                    alcoholic hepatitis       1.00      1.00      1.00         3
             

C:\Users\Ashwin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Ashwin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
Pkl_Filename = "CPS.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(NN, file)

In [15]:
with open(Pkl_Filename, 'rb') as file:  
    CPS_Model = pickle.load(file)

CPS_Model

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('mlpclf',
                 MLPClassifier(hidden_layer_sizes=[100, 100], random_state=0))])

In [23]:
y_pred = CPS_Model.predict_proba(["fever "])
pred = CPS_Model.predict(["itching skin rash"])
print(y_pred.max(), pred,'\n\n')
print(pred)
print (y_pred)

0.2854053102549272 ['fungal infection'] 


['fungal infection']
[[1.78301794e-02 6.11082590e-03 1.77188767e-03 1.37168955e-02
  4.16650123e-03 4.12936400e-02 2.17023398e-03 7.10347906e-02
  3.22433410e-04 3.14899375e-02 1.62658846e-02 1.05039300e-02
  9.85249681e-04 5.31397449e-03 2.15133205e-03 2.47587804e-02
  1.64736620e-02 9.19073944e-03 1.24867348e-02 8.65197861e-03
  8.78354823e-04 2.20512178e-04 1.08422118e-03 1.04015625e-02
  3.64267526e-02 6.04318282e-03 4.43889170e-02 5.84131172e-02
  2.85405310e-01 1.05767633e-02 4.35376233e-03 1.19270312e-03
  3.94563384e-04 4.51950848e-02 1.23073745e-03 1.77589734e-01
  1.56438251e-02 2.10088819e-04 3.66121635e-03]]


In [17]:
MLPClassifier?

Init signature:
MLPClassifier(
    hidden_layer_sizes=(100,),
    activation='relu',
    *,
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='constant',
    learning_rate_init=0.001,
    power_t=0.5,
    max_iter=200,
    shuffle=True,
    random_state=None,
    tol=0.0001,
    verbose=False,
    warm_start=False,
    momentum=0.9,
    nesterovs_momentum=True,
    early_stopping=False,
    validation_fraction=0.1,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
    n_iter_no_change=10,
    max_fun=15000,
)
Docstring:     
Multi-layer Perceptron classifier.

This model optimizes the log-loss function using LBFGS or stochastic
gradient descent.

.. versionadded:: 0.18

Parameters
----------
hidden_layer_sizes : tuple, length = n_layers - 2, default=(100,)
    The ith element represents the number of neurons in the ith
    hidden layer.

activation : {'identity', 'logistic', 'tanh', 'relu'}, default='relu'
    Activation function for the hidden layer.

  